In [1]:
import networkx as nx
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
def Make_Question(n, m, factor=10, Density = 0.75, option = "1-norm", distance = 2):
    nx, ny = (n, m)
    x = np.arange(nx,dtype=float)
    y = np.arange(ny,dtype=float)
    xv, yv = np.meshgrid(x, y)
    Px = np.power(factor,-np.abs(xv-(n-1)/2))
    Py = np.power(factor,-np.abs(yv-(m-1)/2))
    mesh_porbability = np.multiply(Px,Py)
    final_probability = np.multiply(Px,Py)/np.sum(mesh_porbability)
    amount_panel = np.int(Density*n*m)
    allocation = np.random.choice(np.arange(n*m), amount_panel, replace=False, p = final_probability.flatten())
    x_position = allocation // m
    y_position = allocation %  m  
    
    vertices = np.vstack((x_position, y_position)).T
    vertices = vertices[np.lexsort((-vertices[:,0],-vertices[:,1]))]
    
    
    example = vertices[0]
    network = np.zeros((amount_panel,amount_panel))
    for i, vertex in enumerate(vertices):
        example = vertices[i]
        if option == "1-norm":
            # adjacent option
            index = np.where(np.sum(np.abs(vertices-example),axis = 1) == 1)
        elif option == "2-norm":
            # include diag or more
            index = np.where(np.linalg.norm(np.abs(vertices-example),axis = 1) < distance)
        else:
            print("error_input")
        network[i][index] = 1
        
    
    network_adv = np.triu(network, k=1)
    network_adv = network_adv + network_adv.T
    
    
    network_adv = np.pad(network_adv, ((1,0), (1,0)), 'constant', constant_values=((0, 0), (1, 0)))
    network_adv[0,0] = 0
    
    #print(vertices)
    pos = {0: (2*n+0.7,m * 2)} 
    for i, vertex in enumerate(vertices * 2):
        temp_pos = (vertex[0],vertex[1])
        pos.update({i+1: temp_pos})
    return network_adv,pos

In [4]:
def complete_Graph (network,pos):
    # Draw the Directed Graph
    
    plt.figure(1)
    
    G = nx.DiGraph()
    G = nx.from_numpy_array(network,create_using=nx.DiGraph(directed=True))
    
    ##
    
    network_0 = network.copy()
    network_0[1:,1:] = 0
    
    G_0 = nx.DiGraph()
    G_0 = nx.from_numpy_array(network_0,create_using=nx.DiGraph(directed=True))
      
    nx.draw(G_0,pos,with_labels=True)
    
    edge_label = {}
    for j, edge in enumerate(G_0.edges()):
        edge_label.update({(edge[0],edge[1]): str(j)})
        
    graph = nx.draw_networkx_edge_labels(G_0,pos,edge_labels = edge_label)
    
    ##
    
    G_prime, sub_edge_label, sub_graph = sub_Graph (network,pos)
    
    ##
    edge_label.update(sub_edge_label)
    graph.update(sub_graph)
    
    return G, edge_label, graph

In [5]:
def sub_Graph (network,pos):
    
    plt.figure(2)
    
    G_prime = nx.DiGraph()
    G_prime = nx.from_numpy_array(network,create_using=nx.DiGraph(directed=True))
    G_prime.remove_node(0)
    
    
    # Label Edge with following rules:
    # 1. num(BUS)<num(branch)
    # 2. priority(BUS)>priority(branch)
    sub_edge_label = {}
    sub_edge_label_up = {}
    sub_edge_label_down = {}
    sub_edge_label_initial = {}
    
    for j, edge in enumerate(G_prime.edges()):
        if edge[1] > edge[0]:
            sub_edge_label_up.update({(edge[0],edge[1]): str(j+len(pos)-1)})
        else:
            sub_edge_label_down.update({(edge[0],edge[1]): str(j+len(pos)-1)})
        
    
    nx.draw(G_prime,pos,with_labels=True,connectionstyle='arc3, rad = 0.2')
    
    
    sub_edge_label.update(sub_edge_label_initial)
    sub_edge_label.update(sub_edge_label_up)
    sub_edge_label.update(sub_edge_label_down)
    
    sub_graph_initial = nx.draw_networkx_edge_labels(G_prime,pos,edge_labels = sub_edge_label_initial)
    sub_graph_up = nx.draw_networkx_edge_labels(G_prime,pos,edge_labels = sub_edge_label_up, label_pos = 0.65, verticalalignment = 'bottom',horizontalalignment = 'right')
    sub_graph_down = nx.draw_networkx_edge_labels(G_prime,pos,edge_labels = sub_edge_label_down, label_pos = 0.65, verticalalignment = 'top',horizontalalignment = 'left')

    sub_graph = {}
    sub_graph.update(sub_graph_initial)
    sub_graph.update(sub_graph_up)
    sub_graph.update(sub_graph_down)
    
    return G_prime, sub_edge_label, sub_graph

In [6]:
def edge_map (G, compelete_label, sub_edge_label):
    
    plt.figure(3)
    
    complete_edge = np.asarray(G.edges())
    new_map = np.zeros((len(compelete_label),len(compelete_label)))
    #print(complete_edge)
    
    lowest_edge = len(compelete_label) - len(sub_edge_label)
    
    for edge in sub_edge_label:
        
        #leaving edge       
        #map_index_0, _ = np.where(complete_edge[] == edge[0])

                
        map_index_source = np.where(complete_edge[:,0] == edge[1])
        source_edge_index = np.int(compelete_label[(edge[0],edge[1])])
        
        for matched_index in map_index_source[0]:
            
            if (complete_edge[matched_index][0],complete_edge[matched_index][1]) != (edge[1],edge[0]):
                to_edge_index = np.int(compelete_label[(complete_edge[matched_index][0],complete_edge[matched_index][1])])
            
                #if to_edge_index >= lowest_edge:
                  
                new_map[source_edge_index][to_edge_index] = 1
        
        
        
    return new_map, lowest_edge

In [2]:
def make_dictionary(edge_number, Q, N_tree = None):
    position_dictionary = {}
    # set-up a dictionary
    Total = 0
    # x input
    for e in range(edge_number):
        
        for d in range(Q):
            
            multiplier = np.array([e, d + 1])
            
            unit = np.array([Q,1])

            position_dictionary["x_%d_%d"%(e, d + 1)] = int(np.dot(multiplier,unit)) - 1
            
    Total_edge = len(position_dictionary)   
    
    if N_tree is not None:
        for tree, element in enumerate(N_tree):
            position_dictionary["x_tree_%d"%(tree)] = int(Total_edge + tree) - 1
        
    Total = len(position_dictionary)   
    return position_dictionary, Total

In [1]:
def capacity_subtree_limit_generator(pos,lowest_edge_number):
    # the max and min number of subtrees with a capacity equal to t in T 
    # option 1 (disagrd)
    #m_t_u = np.round(0.55 * Q * np.ones(Q))
    #m_t_l = np.round(0.25 * Q * np.ones(Q))
    
    Q = len(pos)-1
    
    #应该作为 Input 
    flag = np.inf
    
    while flag > 3:
        m_t_l, m_t_u = np.sort(np.random.choice(int(Q), 2, replace=False))
        flag = m_t_u - m_t_l
        
    print(m_t_l, m_t_u)
        
    return np.arange(m_t_l + 1, m_t_u + 1 )

## Objective
$$
\sum_{n \in N}x_{n} \cdot \left(\sum_{e \in E} c_e \sum_{d = 1}^{Q} x_{e,d} + p \sum_{e \in E'} c_e \sum_{d = 1}^{Q} (d-1) x_{e,d}\right)
$$

In [8]:
def Objective(Q, edge_number, edge_dictionary, QUBO_matrix_initial, N_tree, basic_cost,flow_cost):
    # Q : # of solar panel node
    # edge_number: Total number of directional edges
    # QUBO_matrix_initial: initialed QUBO (usually zeros matrix)
    # N_tree: # Number of subtree that generated from energy collector node.
    # basic_cost: cost to build the line that afford unit current flow
    # flow_cost: cost to afford extra current flows bigger than unit flow (>1)
    
    
    #the set of capacity types
    #objective

    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)

    ## term 1
    for tree, element in enumerate(N_tree):
        
        for edge in range(edge_number):
            for level in range(Q):
                #print(level,edge)
                index = edge_dictionary['x_%d_%d'%(edge,level + 1)]
                
                tree_index = edge_dictionary['x_tree_%d'%(tree)]
                
                QUBO_matrix[tree_index][index] = basic_cost[edge]


        ## term 2 E\E' are Bus edges == Q
        for edge in range(Q):
            for level in range(Q):
                index = edge_dictionary['x_%d_%d'%(edge,level + 1)]
                
                tree_index = edge_dictionary['x_tree_%d'%(tree)]
                
                QUBO_matrix[tree_index][index] += basic_cost[edge] * flow_cost[edge] * level
    
    return QUBO_matrix

## Constraint 1
\usepackage{mathtools}

$$
\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d} - 1
$$
$$
\Rightarrow (\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d} - 1)^2
$$
$$
\Rightarrow (\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d})^2 - 2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d} +  1
$$
$$
\Rightarrow \sum_{m, i \in \delta^+ (i)} \sum_{n, j \in Q}x_{i,j}x_{m,n} - 2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d} +  1
$$
Since QUBO, leave the constant term out from matrix
$$
\Rightarrow \sum_{m, i \in \delta^+ (i)} \sum_{n, j \in Q}x_{i,j}x_{m,n} - 2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d}
$$

In [1]:
def Constraint_1(Final_map, Q, edge_number, edge_dictionary, compelete_label, QUBO_matrix_initial, Total, penalty = 50):
    # constraint 1
    # QUBO matrix = 2 sum{m,i \in leaving edge}( sum{n,j \in Q}( x_ij*x_mn))     [term 2]
    #             - 3 sum{e \in leaving edge}( sum{d \in Q}( x_ed))              [term 1]
    #             + 1                                                            [term 3]
    #Kick the node 0 from the set of capacity since it does not generate energy

    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)
    len_QUBO_matrix = QUBO_matrix

    for vertex in range(1, Q + 1):
        index = np.hstack(np.asarray(np.where(Final_map[vertex,:])))             
            
        index_matched = []
        for vertex_selected in index:
            
            #print(int(compelete_label[(vertex, vertex_selected)]))
            
            for level in range(Q):
                index_matched.append(edge_dictionary['x_%d_%d'%( int(compelete_label[(vertex, vertex_selected)]), level + 1)])
                
                
              
        #print(index , index_matched)
        var_matched = np.zeros(Total)
        var_matched[index_matched] = 1
        
        temp = np.outer(var_matched,var_matched) - 2 * np.diag(var_matched)
        
        QUBO_matrix += penalty * (temp / np.max(np.abs(temp)))
        #QUBO_matrix += penalty * temp
       
    return QUBO_matrix

## Constraint 2

$$
\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} d x_{e,d}- \sum_{e \in \delta^- (i)} \sum_{d = 1}^{Q}d x_{e,d} - 1
$$
$$
\Rightarrow (\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} d x_{e,d}- \sum_{e \in \delta^- (i)} \sum_{d = 1}^{Q} d x_{e,d} - 1)^2 \\
$$
$$
\Rightarrow (A-B-1)^2
$$
$$
\Rightarrow (A-1)^2 + B^2 -2(A-1)B 
$$
$$
\Rightarrow (A^2 - 2A - 1) + B^2 + 2B -2AB 
$$

Since QUBO, leave the constant term out from matrix
$$
\Rightarrow (A-1)^2 + (B+1)^2 -2AB - 1\\
$$
$$
(A-1)^2\Rightarrow  \sum_{m, i \in \delta^+ (i)} \sum_{n, j \in Q}j n x_{i,j}x_{m,n} - 2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} d x_{e,d}  +  1 
$$
Since QUBO, leave the constant term out from matrix
$$
\Rightarrow  \sum_{m, i \in \delta^+ (i)} \sum_{n, j \in Q}j n x_{i,j}x_{m,n} - 2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} d x_{e,d}
$$
$$
(B+1)^2\Rightarrow  \sum_{m, i \in \delta^- (i)} \sum_{n, j \in Q}j n x_{i,j}x_{m,n} + 2\sum_{e \in \delta^- (i)} \sum_{d = 1}^{Q} d x_{e,d}  +  1 
$$
Since QUBO, leave the constant term out from matrix
$$
\Rightarrow  \sum_{m, i \in \delta^- (i)} \sum_{n, j \in Q}j n x_{i,j}x_{m,n} + 2\sum_{e \in \delta^- (i)} \sum_{d = 1}^{Q} d x_{e,d} 
$$

$$
2AB\Rightarrow  2\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q}d x_{e,d} \sum_{e' \in \delta^- (i)} \sum_{d' = 1}^{Q} d'x_{e',d'}
= 2 \sum_{d = 1}^{Q}  \sum_{d' = 1}^{Q} d d' \sum_{e \in \delta^+ (i)}\sum_{e' \in \delta^- (i)} x_{e,d} x_{e',d'}
=AB+BA$$

In [3]:
def Constraint_2(Final_map, Q, edge_number, edge_dictionary, compelete_label, QUBO_matrix_initial, Total, penalty = 50):
    # constraint 2 
    # QUBO matrix = (sum{e \in leaving edge}( sum{d \in Q}(d * x_ed))   - sum{e \in enter edge}( sum{d \in Q}(d * x_ed)) - 1)^2
    #             = (A-B-1)^2
    #             =(A-1)^2 +(B-1)^2 -2AB + 4B - 1
    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)

    for vertex in range(1, Q + 1):
        
        #print("vertex", vertex)
        
        # term (A-1)^2
        index_out = np.hstack(np.asarray(np.where(Final_map[vertex,:])))     
            
        var_matched_out = np.zeros(Total)
        
        print('OUT')
        print("index_out:", index_out)   
        
        for vertex_selected in index_out:
            
            #### ADD ON
            print("edge", int(compelete_label[(vertex, vertex_selected)]))
            ####
            
            for level in range(Q):
                
                index_matched_out = edge_dictionary['x_%d_%d'%(int(compelete_label[(vertex, vertex_selected)]), level + 1)]
                
                var_matched_out[index_matched_out] = level + 1
        
        # term (B+1)^2
        index_in = np.hstack(np.asarray(np.where(Final_map[:,vertex])))
        
        var_matched_in = np.zeros(Total)
        
        print('IN')
                    
        print("index_in:", index_in)  
        
        for vertex_selected in index_in:
            
            #### ADD ON
            print("edge", int(compelete_label[(vertex_selected, vertex)]))
            ####
            
            for level in range(Q):
                
                index_matched_in = edge_dictionary['x_%d_%d'%(int(compelete_label[(vertex_selected, vertex)]), level + 1)]
                
                var_matched_in[index_matched_in] = level + 1 
        
        # term (A-1)^2
        temp = np.outer(var_matched_out,var_matched_out) - 2 * np.diag(var_matched_out)
        
        # term (B+1)^2
        temp += np.outer(var_matched_in,var_matched_in) + 2 * np.diag(var_matched_in)
            
        # term -2AB
        temp -=  (np.outer(var_matched_in,var_matched_out) + np.outer(var_matched_out,var_matched_in))

      
        QUBO_matrix += penalty * (temp / np.max(np.abs(temp)))
        #QUBO_matrix += penalty * temp
        
    return QUBO_matrix

## Constraint 3

$$
\sum_{d = 1}^{Q} x_{e,d} + \sum_{d = 1}^{Q} x_{e',d} \leq1 \quad such that \quad e*e' = \mathbb{1}
$$

In [7]:
def Constraint_3(Final_map, Q, edge_dictionary, compelete_label, QUBO_matrix_initial, Total, penalty = 50):
    
    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)
    for key in compelete_label.keys():
    #print(key, key[::-1])
        try:
         
            target_edge = np.int(compelete_label[key])
            paired_edge = np.int(compelete_label[key[::-1]])
            
            #print(target_edge,paired_edge)

            var_matched = np.zeros(Total)
            
            for level in range(Q):
                
                index_matched_target = edge_dictionary['x_%d_%d'%(target_edge,level + 1)]
                index_matched_paired = edge_dictionary['x_%d_%d'%(paired_edge,level + 1)]
                
                #print(index_matched_target,index_matched_paired)
                
                var_matched[index_matched_target] = 1
                var_matched[index_matched_paired] = 1

            temp = np.outer(var_matched,var_matched) - np.diag(var_matched)
            
            QUBO_matrix += penalty * (temp / np.max(np.abs(temp)))
            #QUBO_matrix += penalty * temp
            
        except:
            print(key,"next")
            continue
            
        
    return QUBO_matrix
    

## Constraint 4
$$
\sum_{e\in E\backslash E'}\sum^{Q}_{d = 1} x_{e,d} = \sum_{k \in K} N_{k}x_{k}
$$
$$
\Rightarrow (\sum_{e\in E\backslash E'} \sum_{d = 1}^{Q} x_{e,d}- \sum_{k \in K} N_{k}x_{k})^2 \\
$$

$$
\Rightarrow  \sum_{e\in E\backslash E'} \sum_{e^{\prime} \in E\backslash E'} \sum_{d = 1}^{Q} \sum_{t = 1}^{Q} x_{e,d}x_{e^{\prime},t} -  2\sum_{e\in E\backslash E'}\sum_{d = 1}^{Q} \sum_{k \in K}N_{k}x_{e,d}x_{k} + \sum_{k \in K}\sum_{k^{\prime} \in K} N_{k}N_{k^{\prime}}x_{k}x_{k^{\prime}}
$$


In [8]:
def Constraint_4(Q, edge_dictionary, QUBO_matrix_initial, Total, N_k, penalty = 50):
    
    # constraint 4



    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)
    
    len_QUBO_matrix = QUBO_matrix
    
    index_option_matched = []
    
    #print(index_option_matched)
    
    for option_num, option in enumerate(N_k):
    
        index_option_matched.append(edge_dictionary["x_tree_%d"%(option_num)])
    
        
    index_bus_matched = []
    
    
    for Bus_edge in range(Q):         

        for level in range(Q):
                    
            index_bus_matched.append(edge_dictionary['x_%d_%d'%(Bus_edge, level + 1)])

    var_bus_matched = np.zeros(Total)
        
    var_option_matched = np.zeros(Total)
    
    var_bus_matched[ index_bus_matched ] = 1
        
    var_option_matched[ index_option_matched] = N_k
    #print(var_option_matched)

    temp =  np.outer(var_bus_matched, var_bus_matched)        \
            - (np.outer(var_bus_matched, var_option_matched)   \
            + np.outer(var_option_matched, var_bus_matched))    \
            + np.outer(var_option_matched, var_option_matched)
    
    QUBO_matrix += penalty * (temp / np.max(np.abs(temp)))
    #QUBO_matrix += penalty * temp
    
    return QUBO_matrix

## Constraint 5
$$
\sum_{k\in K} x_{tree,k} = 1 \quad \textrm{for } k \in K
$$
$$
\Rightarrow (\sum_{k\in K} x_{tree,k} -1 )^2 \\
$$

$$
\Rightarrow  \sum_{k\in K}\sum_{k^{\prime}\in K} x_{tree,k}x_{tree,k^{\prime}} -  2\sum_{k\in K} x_{tree,k}
$$

In [10]:
def Constraint_5( Q, edge_dictionary, QUBO_matrix_initial, Total, N_k, penalty = 50):
    
    # constraint 4

    #print(penalty)

    QUBO_matrix = np.zeros_like(QUBO_matrix_initial)
    len_QUBO_matrix = QUBO_matrix
    
    index_option_matched = []
    
    for option_num, option in enumerate(N_k):
        
        index_option_matched.append(edge_dictionary["x_tree_%d"%(option_num)])
        
    var_option_matched = np.zeros(Total)
    
    #print(index_option_matched)
        
    var_option_matched[ index_option_matched ] = 1

    temp = np.outer(var_option_matched,var_option_matched) - 2 * np.diag(var_option_matched)
        
    QUBO_matrix += penalty * (temp / np.max(np.abs(temp)))
    
    #QUBO_matrix += penalty * temp
    
    return QUBO_matrix



## contraint 4
need larger penalty $P' = 10^3$.

$$
\Rightarrow P\left( \left\{x_{e,t} \mid  e\in E \backslash E', t\not\in T \right\} \right)
$$

In [11]:
# Ban_map size(edge(E\E'),Q) Full Binary map
def Constraint_6(QUBO_matrix, Q, Ban_map_Bus, penalty = 1000):
    
    #invert gate
    Ban_map_invert = 1 - Ban_map_Bus
    
    QUBO = np.zeros_like(QUBO_matrix)
    
    QUBO[:Q**2, :Q**2] = penalty * np.diag(Ban_map_invert) 
    
    QUBO_matrix += QUBO
    
    # return final martix and a partial penalty matrix with size (e (\in E\E') *Q, e (\in E\E') *Q)
    
    return QUBO_matrix, QUBO



## Quantum return interpretation


In [12]:
def Q_convert(Q, sampleset, edge_number):

    OP = sampleset.to_pandas_dataframe()

    OP = OP.sort_values("energy")
    
    OP_final = OP.to_numpy()

    rank = 0

    OP_opt = OP_final[rank,:].flatten()

    location = np.where(OP_opt[:-2])[0]

    Q = len(pos)-1  

    final_index = location[location < edge_number * Q ]
    
    choice = OP_opt[edge_number * Q : -2]
    
    return final_index, choice

## Check 1

$$
\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} x_{e,d} = 1
$$


In [13]:
def check_1(Final_answer, Final_map, Q, edge_number, edge_dictionary):
    
    print("Now, checking Contraint 1")  
    true_map = []
    
    for vertex in range(1, Q + 1):
        index = np.hstack(np.asarray(np.where(Final_map[vertex,:])))
            
            
        Rhs = 0
        for edge_selected in index:
            for level in range(Q):
                Rhs += Final_answer[edge_dictionary['x_%d_%d'%(int(compelete_label[(vertex,edge_selected)]),level + 1 )]]
        true_map.append(Rhs == 1)
    print("Constraint 1 is ", np.array(true_map).all(), " ===> Constraint 1 is Done.")
    print(" ")

## Check 2

$$
\sum_{e \in \delta^+ (i)} \sum_{d = 1}^{Q} d x_{e,d}- \sum_{e \in \delta^- (i)} \sum_{d = 1}^{Q}d x_{e,d} = 1
$$


In [14]:
def check_2(Final_answer, Final_map, Q, edge_number, edge_dictionary):
    print("Now, checking Contraint 2")    
    true_map = []
    
    for vertex in range(1, Q + 1):
        index_out = np.hstack(np.asarray(np.where(Final_map[vertex,:])))
            
            
        Rhs = 0
        for edge_selected in index_out:
            for level in range(Q):
                Rhs += (level + 1) * Final_answer[edge_dictionary['x_%d_%d'%(int(compelete_label[(vertex,edge_selected)]),level + 1 )]]
        
        index_in = np.hstack(np.asarray(np.where(Final_map[:,vertex])))
        
        for edge_selected in index_in:
            for level in range(Q):
                Rhs -= (level + 1) * Final_answer[edge_dictionary['x_%d_%d'%(int(compelete_label[(edge_selected, vertex)]),level + 1 )]]
        
        true_map.append(Rhs == 1)
        #print("vertex ", vertex , " ", Rhs)
    print("Constraint 2 is ", np.array(true_map).all(), " ===> Constraint 2 is Done.")
    print(" ")

## Check 3

$$
\sum_{e\in E\backslash E'}\sum^{Q}_{d = 1} x_{e,d} = N_{k}x_{k} \quad \textrm{for } k \in K
$$

In [15]:
def check_3(Final_answer, Final_map, Q, edge_number, edge_dictionary, m_t_u, m_t_l):
    print("Now, checking Contraint 3")    
    true_map = []
    
    for level in range(Q):
        result = 0
        
        for edge_selected in range(Q):
            result += Final_answer[edge_dictionary['x_%d_%d'%( edge_selected ,level)]] 
            
        true_map.append(m_t_l[level] <= result <= m_t_u[level])
        
    print("Constraint 3 is ", np.array(true_map).all(), " ===> Constraint 3 is Done.")
    print(" ")    

In [16]:
def show_graph(QUBO):
    matfig = plt.figure(figsize=(6,6)) 
    plt.matshow(QUBO,fignum=matfig.number)
    plt.colorbar()
    plt.show()